# Exploratory Data Analysis

Use this notebook to inspect raw data, check quality, and prototype features.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100

# TODO: set your raw data path
raw_path = '../data/raw/example.csv'

try:
    df = pd.read_csv(raw_path)
    display(df.head())
    display(df.describe(include='all'))
    sns.pairplot(df.sample(min(len(df), 200)))
    plt.show()
except FileNotFoundError:
    print(f'Place a CSV at {raw_path} to begin exploring.')
